## Import all needed libraries


In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

## 1. LOAD DATA

In [2]:
print("Loading silver price data...")
ticker = "SI=F"
df = yf.download(ticker, start="2020-01-01", progress=False)
df = df[['Close']].copy()
df.columns = ['Price']
df = df.dropna().reset_index()

print(f"Data shape: {df.shape}")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"\nRecent prices (last 10 days):")
print(df.tail(10)[['Date', 'Price']].to_string(index=False))

Loading silver price data...
Data shape: (1459, 2)
Date range: 2020-01-02 00:00:00 to 2025-10-17 00:00:00

Recent prices (last 10 days):
      Date     Price
2025-10-06 48.082001
2025-10-07 47.179001
2025-10-08 48.655998
2025-10-09 46.849998
2025-10-10 46.938000
2025-10-13 50.130001
2025-10-14 50.313999
2025-10-15 51.073002
2025-10-16 53.022999
2025-10-17 49.863998


## 2. ANALYZE RECENT TREND

In [9]:
print("ANALYZING RECENT PRICE MOVEMENT")

recent_30 = df.tail(30)
recent_7 = df.tail(7)
recent_3 = df.tail(3)

avg_30 = recent_30['Price'].mean()
avg_7 = recent_7['Price'].mean()
avg_3 = recent_3['Price'].mean()

print(f"\n30-day average: ${avg_30:.2f}")
print(f"7-day average:  ${avg_7:.2f}")
print(f"3-day average:  ${avg_3:.2f}")
print(f"Current price:  ${df['Price'].iloc[-1]:.2f}")

volatility_30 = recent_30['Price'].std()
volatility_7 = recent_7['Price'].std()

print(f"\n30-day volatility: ${volatility_30:.2f}")
print(f"7-day volatility:  ${volatility_7:.2f}")

ANALYZING RECENT PRICE MOVEMENT

30-day average: $45.57
7-day average:  $49.74
3-day average:  $51.32
Current price:  $49.86

30-day volatility: $3.37
7-day volatility:  $2.21


## 3. APPROACH 1: EXPONENTIAL SMOOTHING (gives more weight to recent data)

In [10]:
print("MODEL 1: EXPONENTIAL WEIGHTED MOVING AVERAGE")

def exponential_smoothing_forecast(series, alpha=0.3):
    """Simple exponential smoothing with custom alpha"""
    result = [series.iloc[0]]
    for i in range(1, len(series)):
        result.append(alpha * series.iloc[i] + (1 - alpha) * result[i-1])
    return result

# Try different alpha values (higher = more weight on recent data)
alphas = [0.2, 0.4, 0.6, 0.8]
best_alpha = None
best_mae = float('inf')

print("\nTesting different smoothing parameters on last 10 days:")
for alpha in alphas:
    train = df['Price'].iloc[:-10]
    test = df['Price'].iloc[-10:]

    smoothed = exponential_smoothing_forecast(train, alpha)
    last_smoothed = smoothed[-1]

    # Forecast next 10 values
    forecast = [last_smoothed]
    for _ in range(9):
        forecast.append(alpha * test.iloc[len(forecast)-1] + (1-alpha) * forecast[-1])

    mae = mean_absolute_error(test, forecast)
    print(f"  Alpha={alpha}: MAE=${mae:.2f}")

    if mae < best_mae:
        best_mae = mae
        best_alpha = alpha

print(f"\nBest alpha: {best_alpha}")

# Generate forecast with best alpha
smoothed_full = exponential_smoothing_forecast(df['Price'], best_alpha)
next_pred_ema = best_alpha * df['Price'].iloc[-1] + (1-best_alpha) * smoothed_full[-1]

print(f"Next day prediction: ${next_pred_ema:.2f}")

MODEL 1: EXPONENTIAL WEIGHTED MOVING AVERAGE

Testing different smoothing parameters on last 10 days:
  Alpha=0.2: MAE=$1.90
  Alpha=0.4: MAE=$1.54
  Alpha=0.6: MAE=$1.49
  Alpha=0.8: MAE=$1.43

Best alpha: 0.8
Next day prediction: $49.97


## 4. APPROACH 2: ADAPTIVE WEIGHTED AVERAGE (recent days get exponentially more weight)

In [11]:
print("MODEL 2: ADAPTIVE WEIGHTED AVERAGE")

def adaptive_weighted_forecast(series, window=7, decay=0.5):
    """Weighted average with exponential decay - recent days matter more"""
    recent = series.iloc[-window:].values
    weights = np.array([decay ** (window - i - 1) for i in range(window)])
    weights = weights / weights.sum()
    return np.sum(recent * weights)

windows = [5, 7, 10, 14]
print("\nTesting different lookback windows:")

best_window = None
best_decay = None
best_mae = float('inf')

for window in windows:
    for decay in [0.3, 0.5, 0.7]:
        predictions = []
        actuals = []

        # Test on last 10 days
        for i in range(len(df) - 10, len(df)):
            pred = adaptive_weighted_forecast(df['Price'].iloc[:i], window, decay)
            predictions.append(pred)
            actuals.append(df['Price'].iloc[i])

        mae = mean_absolute_error(actuals, predictions)
        print(f"  Window={window:2d}, Decay={decay}: MAE=${mae:.2f}")

        if mae < best_mae:
            best_mae = mae
            best_window = window
            best_decay = decay

print(f"\nBest parameters: Window={best_window}, Decay={best_decay}")
next_pred_adaptive = adaptive_weighted_forecast(df['Price'], best_window, best_decay)
print(f"Next day prediction: ${next_pred_adaptive:.2f}")

MODEL 2: ADAPTIVE WEIGHTED AVERAGE

Testing different lookback windows:
  Window= 5, Decay=0.3: MAE=$1.46
  Window= 5, Decay=0.5: MAE=$1.51
  Window= 5, Decay=0.7: MAE=$1.55
  Window= 7, Decay=0.3: MAE=$1.46
  Window= 7, Decay=0.5: MAE=$1.51
  Window= 7, Decay=0.7: MAE=$1.55
  Window=10, Decay=0.3: MAE=$1.46
  Window=10, Decay=0.5: MAE=$1.51
  Window=10, Decay=0.7: MAE=$1.58
  Window=14, Decay=0.3: MAE=$1.46
  Window=14, Decay=0.5: MAE=$1.51
  Window=14, Decay=0.7: MAE=$1.61

Best parameters: Window=7, Decay=0.3
Next day prediction: $50.61


## 5. APPROACH 3: MOMENTUM-BASED FORECAST

In [12]:
print("MODEL 3: MOMENTUM-BASED FORECAST")

def momentum_forecast(series, momentum_window=3, trend_window=7):
    """Use recent momentum to project forward"""
    recent = series.iloc[-momentum_window:]
    momentum = (recent.iloc[-1] - recent.iloc[0]) / momentum_window

    # Also consider longer trend
    longer = series.iloc[-trend_window:]
    trend = (longer.iloc[-1] - longer.iloc[0]) / trend_window

    # Weighted combination
    combined_momentum = 0.7 * momentum + 0.3 * trend

    return series.iloc[-1] + combined_momentum

# Test different windows
print("\nTesting different momentum windows:")
best_mom_mae = float('inf')
best_mom_window = None
best_trend_window = None

for mom_win in [2, 3, 5]:
    for trend_win in [5, 7, 10]:
        predictions = []
        actuals = []

        for i in range(len(df) - 10, len(df)):
            pred = momentum_forecast(df['Price'].iloc[:i], mom_win, trend_win)
            predictions.append(pred)
            actuals.append(df['Price'].iloc[i])

        mae = mean_absolute_error(actuals, predictions)
        print(f"  Momentum={mom_win}, Trend={trend_win}: MAE=${mae:.2f}")

        if mae < best_mom_mae:
            best_mom_mae = mae
            best_mom_window = mom_win
            best_trend_window = trend_win

print(f"\nBest parameters: Momentum={best_mom_window}, Trend={best_trend_window}")
next_pred_momentum = momentum_forecast(df['Price'], best_mom_window, best_trend_window)
print(f"Next day prediction: ${next_pred_momentum:.2f}")

MODEL 3: MOMENTUM-BASED FORECAST

Testing different momentum windows:
  Momentum=2, Trend=5: MAE=$1.69
  Momentum=2, Trend=7: MAE=$1.66
  Momentum=2, Trend=10: MAE=$1.66
  Momentum=3, Trend=5: MAE=$1.59
  Momentum=3, Trend=7: MAE=$1.54
  Momentum=3, Trend=10: MAE=$1.54
  Momentum=5, Trend=5: MAE=$1.54
  Momentum=5, Trend=7: MAE=$1.49
  Momentum=5, Trend=10: MAE=$1.49

Best parameters: Momentum=5, Trend=10
Next day prediction: $49.88


## 6. APPROACH 4: NAIVE METHODS (sometimes best during volatility)

In [13]:
print("MODEL 4: NAIVE FORECASTING METHODS")

# Naive forecast 1: Last value
naive_last = df['Price'].iloc[-1]
print(f"1. Last Value:           ${naive_last:.2f}")

# Naive forecast 2: Average of last 3 days
naive_avg3 = df['Price'].iloc[-3:].mean()
print(f"2. 3-day Average:        ${naive_avg3:.2f}")

# Naive forecast 3: Median of last 5 days (robust to outliers)
naive_median5 = df['Price'].iloc[-5:].median()
print(f"3. 5-day Median:         ${naive_median5:.2f}")

# Naive forecast 4: Weighted recent (60% yesterday, 30% day before, 10% 3 days ago)
naive_weighted = 0.6 * df['Price'].iloc[-1] + 0.3 * df['Price'].iloc[-2] + 0.1 * df['Price'].iloc[-3]
print(f"4. Weighted Recent:      ${naive_weighted:.2f}")

# Test which naive method works best
test_size = 10
naive_methods = {
    'Last Value': lambda s: s.iloc[-1],
    '3-day Average': lambda s: s.iloc[-3:].mean(),
    '5-day Median': lambda s: s.iloc[-5:].median(),
    'Weighted Recent': lambda s: 0.6 * s.iloc[-1] + 0.3 * s.iloc[-2] + 0.1 * s.iloc[-3]
}

print("\nTesting on last 10 days:")
for name, method in naive_methods.items():
    predictions = []
    actuals = []
    for i in range(len(df) - test_size, len(df)):
        pred = method(df['Price'].iloc[:i])
        predictions.append(pred)
        actuals.append(df['Price'].iloc[i])
    mae = mean_absolute_error(actuals, predictions)
    print(f"  {name:20s}: MAE=${mae:.2f}")

MODEL 4: NAIVE FORECASTING METHODS
1. Last Value:           $49.86
2. 3-day Average:        $51.32
3. 5-day Median:         $50.31
4. Weighted Recent:      $50.93

Testing on last 10 days:
  Last Value          : MAE=$1.40
  3-day Average       : MAE=$1.50
  5-day Median        : MAE=$1.62
  Weighted Recent     : MAE=$1.47


## 7. ENSEMBLE OF BEST METHODS

In [16]:
print("ENSEMBLE FORECAST")

# Combine the predictions with equal weight (or optimize weights)
all_predictions = [
    next_pred_ema,
    next_pred_adaptive,
    next_pred_momentum,
    naive_last,
    naive_avg3,
    naive_median5,
    naive_weighted
]

# Simple average
ensemble_simple = np.mean(all_predictions)

# Median (more robust to outliers)
ensemble_median = np.median(all_predictions)

# Trimmed mean (remove highest and lowest)
sorted_preds = sorted(all_predictions)
ensemble_trimmed = np.mean(sorted_preds[1:-1])

print(f"\nAll individual predictions:")
print(f"  EMA:                ${next_pred_ema:.2f}")
print(f"  Adaptive Weighted:  ${next_pred_adaptive:.2f}")
print(f"  Momentum:           ${next_pred_momentum:.2f}")
print(f"  Naive Last:         ${naive_last:.2f}")
print(f"  Naive 3-day Avg:    ${naive_avg3:.2f}")
print(f"  Naive 5-day Median: ${naive_median5:.2f}")
print(f"  Naive Weighted:     ${naive_weighted:.2f}")

print(f"\nEnsemble predictions:")
print(f"  Simple Average:     ${ensemble_simple:.2f}")
print(f"  Median:             ${ensemble_median:.2f}")
print(f"  Trimmed Mean:       ${ensemble_trimmed:.2f}")

current_price = df['Price'].iloc[-1]
print(f"\n")
print(f"CURRENT PRICE: ${current_price:.2f}")
print(f"RECOMMENDED FORECAST (Trimmed Mean): ${ensemble_trimmed:.2f}")
print(f"Change: ${ensemble_trimmed - current_price:+.2f} ({(ensemble_trimmed - current_price)/current_price * 100:+.2f}%)")

ENSEMBLE FORECAST

All individual predictions:
  EMA:                $49.97
  Adaptive Weighted:  $50.61
  Momentum:           $49.88
  Naive Last:         $49.86
  Naive 3-day Avg:    $51.32
  Naive 5-day Median: $50.31
  Naive Weighted:     $50.93

Ensemble predictions:
  Simple Average:     $50.41
  Median:             $50.31
  Trimmed Mean:       $50.34


CURRENT PRICE: $49.86
RECOMMENDED FORECAST (Trimmed Mean): $50.34
Change: $+0.48 (+0.96%)


In [17]:
print("IMPORTANT NOTES")
print("""
1. During extreme volatility (like recent spike from $50 to $53+),
   traditional ML models fail because they've never seen this pattern.

2. Time series methods (especially naive methods) often work better
   during regime changes because they:
   - Don't rely on historical patterns
   - Give most weight to very recent data
   - Are more adaptive to sudden changes

3. The trimmed mean ensemble removes extreme predictions and provides
   a more stable forecast.

4. Given the massive drop from $53 to $49.86 in one day, the market
   is showing high volatility. Consider:
   - Using the median forecast (most robust)
   - Shortening prediction horizon to hours, not days
   - Adding external data (USD index, inflation news, etc.)

5. For trading decisions, focus on ranges rather than point estimates:
   - Likely range: $48 - $52
   - Watch for continued volatility
""")

IMPORTANT NOTES

1. During extreme volatility (like recent spike from $50 to $53+), 
   traditional ML models fail because they've never seen this pattern.

2. Time series methods (especially naive methods) often work better 
   during regime changes because they:
   - Don't rely on historical patterns
   - Give most weight to very recent data
   - Are more adaptive to sudden changes

3. The trimmed mean ensemble removes extreme predictions and provides
   a more stable forecast.

4. Given the massive drop from $53 to $49.86 in one day, the market
   is showing high volatility. Consider:
   - Using the median forecast (most robust)
   - Shortening prediction horizon to hours, not days
   - Adding external data (USD index, inflation news, etc.)
   
5. For trading decisions, focus on ranges rather than point estimates:
   - Likely range: $48 - $52
   - Watch for continued volatility

